In [2]:
%matplotlib inline
from import_all import *
import visa
rm = visa.ResourceManager("/Library/Frameworks/Visa.framework/VISA")
print rm.list_resources()
my_instrument = rm.open_resource('GPIB0::13::INSTR')
print(my_instrument.query("*IDN?"))

(u'GPIB0::13::INSTR', u'GPIB0::15::INSTR', u'GPIB0::22::INSTR')
KEITHLEY INSTRUMENTS INC.,MODEL 2400,1386470,C32   Oct  4 2010 14:20:11/A02  /S/K



In [41]:
# JPN manual p.10-17
import time 

def iv_sweep(v_start, v_end, v_step, sdel=0.05, cmpl=0.1, nplc=0.1):
    """do I-V sweep on Keitheley 2400 source meter.
    
    Args:
    v_start: float, start voltage [V]
    v_end: float, end voltage [V]
    v_step: float, voltage steps [V]
    sdel: float, source delay [s]
    cmpl: float, compliance current [A]
    nplc: integer, Number of Power Line cycles (integration time)
        nplc = 1 takes 16.67 (1/60) msec in theory
    
    Ret:
    output: pd.DataFrame with the columns of [voltage, , , , ]
    """
    s_v_start = str(v_start)
    s_v_end = str(v_end)
    s_v_step = str(v_step)
    s_sdel = str(sdel)
    s_cmpl = str(cmpl)
    
    trg_cnt = round(abs(v_start - v_end)/v_step + 1)
    s_trg_cnt = str(trg_cnt)
    
    s_nplc = str(nplc)
    
    sweep_time = cal_step_time(sdel, nplc) * trg_cnt
    
    print 'est. scan time = %d [s]' %sweep_time
    
    inpt = my_instrument.write('*RST') #reset
    inpt = my_instrument.write(':SOUR:CLE:AUTO ON') #set auto out-put off
    inpt = my_instrument.write(':SENS:FUNC \'CURR:DC\'') # sense DC current
    inpt = my_instrument.write(':SENS:CURR:DC:NPLCycles ' + s_nplc)
    inpt = my_instrument.write(':SENS:CURR:PROT ' + s_cmpl) #set compliance
    inpt = my_instrument.write(':SOUR:VOLT:START ' + s_v_start) #set votage start
    inpt = my_instrument.write(':SOUR:VOLT:STOP ' + s_v_end) #set voltage end
    inpt = my_instrument.write(':SOUR:VOLT:STEP ' + s_v_step) #set voltage step
    inpt = my_instrument.write(':SOUR:VOLT:MODE SWE') #set sweep mode
    inpt = my_instrument.write(':TRIG:COUN ' + s_trg_cnt) #set triger count = (start-end)/step + 1
    inpt = my_instrument.write(':SOUR:DEL ' + s_sdel) #set source delay
        
    my_instrument.timeout = int(sweep_time * 1000 * 2)
    #[ms], "predicted sweep time" * 2 to be in the safer side
    
    buffer_out = my_instrument.query('READ?') #start sweep
    
    return buffer_out


def cal_step_time(sdel, nplc, tdel=0, freq=60.):
    """calculate measurment time in each step.
    
    See Appendex A of the KE2400 manual
    
    """
    TRIG_WAIT = 255e-6 #[s]
    SOURCE_SET = 50e-6 #[s]
    FARMWAER_OVERHEAD = 1.8e-3 #[s]
    
    source_on = SOURCE_SET + sdel + 3 * (nplc * float(1/freq) + 185e-6) + FARMWAER_OVERHEAD
    
    total_time = TRIG_WAIT + tdel + source_on
    
    return total_time
    


In [50]:
t = iv_sweep(-1,2,0.01, sdel=.01, nplc=.1)
print t

0.017405
est. scan time = 5 [s]
-1.000000E+00,+1.920603E-08,+9.910000E+37,+6.462960E+04,+2.150800E+04,-9.900000E-01,+5.513380E-08,+9.910000E+37,+6.462961E+04,+2.150800E+04,-9.800000E-01,-1.059801E-08,+9.910000E+37,+6.462962E+04,+2.150800E+04,-9.700000E-01,-2.894192E-08,+9.910000E+37,+6.462964E+04,+2.150800E+04,-9.600000E-01,-9.907985E-08,+9.910000E+37,+6.462965E+04,+2.150800E+04,-9.500000E-01,-5.086892E-08,+9.910000E+37,+6.462966E+04,+2.150800E+04,-9.400000E-01,+3.736208E-09,+9.910000E+37,+6.462968E+04,+2.150800E+04,-9.300000E-01,-1.922943E-08,+9.910000E+37,+6.462970E+04,+2.150800E+04,-9.200000E-01,-4.151030E-08,+9.910000E+37,+6.462971E+04,+2.150800E+04,-9.100000E-01,-1.012547E-07,+9.910000E+37,+6.462972E+04,+2.150800E+04,-9.000000E-01,-3.707899E-08,+9.910000E+37,+6.462973E+04,+2.150800E+04,-8.900000E-01,+8.980630E-09,+9.910000E+37,+6.462975E+04,+2.150800E+04,-8.800000E-01,+6.210525E-09,+9.910000E+37,+6.462976E+04,+2.150800E+04,-8.700000E-01,-2.457646E-08,+9.910000E+37,+6.462977E+04,+2

In [5]:
print t

-1.000000E+00,+8.370902E-08,+9.910000E+37,+6.259925E+04,+2.150800E+04,-9.000000E-01,-7.228717E-08,+9.910000E+37,+6.259930E+04,+2.150800E+04,-8.000000E-01,+6.215762E-08,+9.910000E+37,+6.259936E+04,+2.150800E+04,-7.000000E-01,-5.059219E-08,+9.910000E+37,+6.259941E+04,+2.150800E+04,-6.000000E-01,+2.375623E-08,+9.910000E+37,+6.259946E+04,+2.150800E+04,-5.000000E-01,-1.766233E-08,+9.910000E+37,+6.259951E+04,+2.150800E+04,-4.000000E-01,+8.381363E-09,+9.910000E+37,+6.259957E+04,+2.150800E+04,-3.000000E-01,-5.316163E-09,+9.910000E+37,+6.259962E+04,+2.150800E+04,-2.000000E-01,-7.620068E-08,+9.910000E+37,+6.259967E+04,+2.150800E+04,-1.000000E-01,+8.793585E-08,+9.910000E+37,+6.259973E+04,+2.150800E+04,+0.000000E+00,+5.827533E-08,+9.910000E+37,+6.259979E+04,+2.150800E+04,+1.000000E-01,-6.803054E-08,+9.910000E+37,+6.259984E+04,+2.150800E+04,+2.000000E-01,+7.557164E-08,+9.910000E+37,+6.259989E+04,+2.150800E+04,+3.000000E-01,-8.129407E-08,+9.910000E+37,+6.259995E+04,+2.150800E+04,+4.000000E-01,+9.440

In [ ]:
    """
    time.sleep(3)
    while True:
        time.sleep(1)
        opr = my_instrument.query(':STAT:OPER:EVEN?')
        reg = read_register(opr, 16)
        print reg
        if reg[-11]=='1': #if "idle"(B11) is on 
            print 'scan completed'
            break
            
    buffer_out = my_instrument.query(':FETC?')
    """
    